<a href="https://colab.research.google.com/github/BrunoGallani/projeto_PLN_UFABC/blob/main/Projeto_PLN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **EQUIPE**

---

**Integrante 01:**

`Bruno Augusto Soares Gallani - RA 11202230282`

**Integrante 02:**

`Gabriel Cipriano Gomes dos Santos - RA 11202111302`

**Integrante 03:**

`Rafael de Souza Coelho - RA 11202232212`

## **GRANDE MODELO DE LINGUAGEM (*Large Language Model - LLM*)**

---

**LLM**: Gemini

**Link para a documentação oficial**: https://ai.google.dev/gemini-api/docs



## **DESCRIÇÃO**
---

Este projeto consiste na implementação de um sistema de Perguntas e Respostas (Q&A) no Google Colab, utilizando o framework **LangChain** em conjunto com o **Large Language Model (LLM) Gemini-2.5-flash**. O objetivo é fornecer respostas precisas a consultas sobre o mercado regulado de carbono, baseando-se em um corpus de informações obtido a partir de um documento PDF da CVM (Comissão de Valores Mobiliários), acessado via <a href="https://www.gov.br/investidor/pt-br/educacional/publicacoes-educacionais/cvm-sustentavel/faq-mercado-de-carbono-set-2025-cvm.pdf/">URL</a>.

Para tal, foram aplicadas as seguintes técnicas de **Processamento de Linguagem Natural** (PLN):

*   **Similaridade de textos:** utilizada para recuperar os trechos mais relevantes do documento que melhor se correlacionam com a consulta do usuário, garantindo que o LLM tenha acesso ao contexto mais apropriado para gerar sua resposta.

*   **Sistemas de perguntas e respostas:** a funcionalidade central do projeto, permitindo que o LLM responda a questões complexas com base exclusivamente no contexto recuperado, sem recorrer a conhecimentos externos.

*   **Sumarização de textos (implícita):** o LLM sintetiza e condensa as informações relevantes dos trechos recuperados para formular respostas concisas, objetivas e alinhadas ao tom institucional da CVM, sem inventar informações ausentes do contexto.

## **IMPLEMENTAÇÃO**
---

### Base para desenvolvimento

In [ ]:
!pip install chromadb pypdf sentence-transformers umap-learn helper_utils langchain_text_splitters langchain_google_genai langchain_core langchain

In [2]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from pypdf import PdfReader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_text_splitters import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
import umap
from google.colab import userdata
import requests
from IPython.display import display, Markdown

In [3]:
CHAVE_API_GOOGLE = userdata.get("GEMINI_KEY")

modelo_google_langchain = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key = CHAVE_API_GOOGLE)

### Leitura/Tratamento Fonte Informação

In [4]:
response = requests.get(url="https://www.gov.br/investidor/pt-br/educacional/publicacoes-educacionais/cvm-sustentavel/faq-mercado-de-carbono-set-2025-cvm.pdf/")
with open('mercado_regulado_carbono_cvm.pdf', 'wb') as f:
    f.write(response.content)

In [5]:
faq_mercado_carbono = PdfReader("/content/mercado_regulado_carbono_cvm.pdf")

In [6]:
texto_faq_mercado_carbono = [pagina.extract_text().strip() for pagina in faq_mercado_carbono.pages if pagina.extract_text().strip()]

### Alocação de Token/*Embedding*

In [7]:
divide_caracteres = RecursiveCharacterTextSplitter(separators=["\n\n","\n",". "," ",""], chunk_size =1000, chunk_overlap = 0 )

In [8]:
texto_caracteres_divididos = divide_caracteres.split_text("\n\n".join(texto_faq_mercado_carbono))

In [9]:
texto_caracteres_divididos[10]

'cumprimento dos compromissos internacionais do Brasil, especialmente aqueles \nassumidos no Acordo de Paris . Ao estabelecer um mercado de carbono \nregulado, o país cria um mecanismo econômico para atingir as metas de redução \nde emissões de GEE para 2035, previstas na Contribuição Nacionalmente \nDeterminada (NDC), que estabelece uma redução de 59% a 67% em \ncomparação a indicadores de 2005 4, rumo a um modelo de desenvolvimento \nsustentável. \n12 - Como o mercado voluntário se relaciona com o SBCE? \nDe acordo com a Lei n° 15.042/2024, o mercado voluntário  é o ambiente \ncaracterizado por transações de créditos de carbono ou ativos integrantes do \nSBCE de forma espontânea , onde os participantes negociam esses créditos \npara compensar suas próprias emissões sem uma obrigação legal específica  e \nsem gerar ajustes na contabilidade nacional de emissões. \nJá as entidades sujeitas ao cumprimento de limites de emissão estabelecidos'

In [ ]:
divide_token = SentenceTransformersTokenTextSplitter(chunk_overlap=20, tokens_per_chunk=256)

In [11]:
texto_token = []
for texto in texto_caracteres_divididos:
  texto_token += divide_token.split_text(texto)

In [12]:
len(texto_token)

22

In [13]:
texto_token

['r set / 2025 gefis / soi faq – mercado regulado de carbono',
 'faq – mercado regulado de carbono ( set / 2025 ) 2 faq – mercado regulado de carbono 1 – o que e o sistema brasileiro de comercio de emissoes de gases de efeito estufa ( sbce )? o sistema brasileiro de comercio de emissoes de gases de efeito estufa ( sbce ) e um ambiente regulado que funciona sob um regime de controle das emissoes de gases de efeito estufa ( gee ) e viabiliza a negociacao de ativos que representam o direito de emissao, bem como a reducao ou remocao de gee. ele atuara como uma ferramenta para reduzir as emissoes de gee, com vistas a promover o desenvolvimento sustentavel, a equidade climatica e estimular inovacoes tecnologicas de baixo carbono. o sbce funcionara no modelo de " cap - and - trade " : e estabelecido um limit',
 'no modelo de " cap - and - trade " : e estabelecido um limite total ( o " cap " ) de emissoes para cada setor ou empresa, os quais podem comprar e vender entre si ( o “ trade ” ) os a

In [ ]:
embedding_function = SentenceTransformerEmbeddingFunction()

### Criação do *db*

In [15]:
banco_dados_vetorial_client = chromadb.Client()
colecao_chroma_carbono = banco_dados_vetorial_client.create_collection("mercado-carbono", embedding_function=embedding_function,)

In [16]:
ids = [str(i) for i in range(len(texto_token))]

In [17]:
colecao_chroma_carbono.add(ids=ids, documents=texto_token)

In [18]:
colecao_chroma_carbono.count()

22

### Teste *retrieval db*

In [19]:
query = "O mercado de carbono é bom?"

In [20]:
resultado_query = colecao_chroma_carbono.query(query_texts=[query], n_results = 5)

In [21]:
resultado_query["documents"][0]

['##do e o publico informados sobre o desenvolvimento do mercado de carbono no brasil. faq – mercado regulado de carbono ( set / 2025 ) 7',
 'estamos atentos aos desafios e as particularidades desse novo mercado. acompanharemos a criacao do novo orgao gestor e contribuiremos para a criacao de um arcabouco regulatorio robusto. alem disso, como e de praxe, a autarquia mantera o dialogo continuo com o mercado e a sociedade visando a construcao e ao aprimoramento das normas.',
 'os ativos a partir do momento que passam a ser negociados em mercados organizados. da mesma forma, os creditos de carbono, do mercado voluntario, tambem sao valores mobiliarios quando negociados no mercado financeiro e de capitais, conforme art. 2, inciso x da lei n° 6. 385 / 762. 5 - qual o papel da cvm nesse novo mercado? o art. 14 da lei n° 15. 042 / 2024 diz que “ os ativos integrantes do sbce e os creditos de carbono, quando negociados no mercado financeiro e de capitais, sao valores mobiliarios sujeitos ao re

### Enriquecimento de Prompt

In [33]:
def gerar_multiplas_queries(query, modelo=modelo_google_langchain):

    prompt = f"""
    Você é um analista senior de finanças aplicado ao mercado de carbono, os clientes de sua empresa querem entender sobre o ramo.
    Para o cenário dado, proponha até cinco perguntas que ajude os clientes a achar a informação que precisam.
    Retorne questões concisas, em topicos simples e de escopo abrangente, evitando alocar mais de um assunto no mesmo tópico.
    Garanta que cada questão esteja completa e diretamente relacionada ao cenario dado.
    Questões separadas apenas por \n no final de cada pergunta e não devem ter numeração e nenhum outro marcador no começo da pergunta.

    Consulta do usuário: '{query}'
    """

    resposta = modelo.invoke(prompt)

    return resposta

In [34]:
query = "Mercado Carbono"
resultado = gerar_multiplas_queries(query)
original_e_augmented = [query] + resultado.content.split("\n")
original_e_augmented_refinado = set()
for resultado in original_e_augmented:
    tratado = resultado.replace("*","").strip()
    if len(tratado) > 0 and "?" in tratado:
        original_e_augmented_refinado.add(tratado)

In [35]:
original_e_augmented_refinado

{'Como a precificação e a volatilidade dos créditos de carbono são determinadas e quais fatores as influenciam?',
 'O que é o mercado de carbono e qual sua finalidade principal?',
 'Quais são as oportunidades de investimento e os riscos financeiros associados à participação no mercado de carbono?',
 'Quais são as tendências regulatórias e de mercado que podem impactar o futuro do mercado de carbono?',
 'Quais são os principais tipos de mercados de carbono (regulados e voluntários) e como eles se diferenciam?'}

### Recuperação de trechos do documento relevantes

In [36]:
documentos_rag = colecao_chroma_carbono.query(query_texts=list(original_e_augmented_refinado), n_results = 5)["documents"]

In [37]:
documentos_unicos = set()
for documentos in documentos_rag:
  for documento in documentos:
    documentos_unicos.add(documento)

In [38]:
documentos_unicos

{'##do e o publico informados sobre o desenvolvimento do mercado de carbono no brasil. faq – mercado regulado de carbono ( set / 2025 ) 7',
 '##izacao desses ativos no mercado de capitais. vamos atuar na transparencia, integridade e seguranca das operacoes visando assegurar o desenvolvimento eficiente deste mercado e a protecao ao investidor. 6 - como a cvm vai assegurar o funcionamento ordenado e a transparencia das negociacoes?',
 'cumprimento dos compromissos internacionais do brasil, especialmente aqueles assumidos no acordo de paris. ao estabelecer um mercado de carbono regulado, o pais cria um mecanismo economico para atingir as metas de reducao de emissoes de gee para 2035, previstas na contribuicao nacionalmente determinada ( ndc ), que estabelece uma reducao de 59 % a 67 % em comparacao a indicadores de 2005 4, rumo a um modelo de desenvolvimento sustentavel. 12 - como o mercado voluntario se relaciona com o sbce? de acordo com a lei n° 15. 042 / 2024, o mercado voluntario e o

### Uso dos trechos relevantes recuperados como contexto para a resposta do LLM

In [39]:
contexto_formatado = "\n".join(list(documentos_unicos))
contexto_formatado

'faq – mercado regulado de carbono ( set / 2025 ) 2 faq – mercado regulado de carbono 1 – o que e o sistema brasileiro de comercio de emissoes de gases de efeito estufa ( sbce )? o sistema brasileiro de comercio de emissoes de gases de efeito estufa ( sbce ) e um ambiente regulado que funciona sob um regime de controle das emissoes de gases de efeito estufa ( gee ) e viabiliza a negociacao de ativos que representam o direito de emissao, bem como a reducao ou remocao de gee. ele atuara como uma ferramenta para reduzir as emissoes de gee, com vistas a promover o desenvolvimento sustentavel, a equidade climatica e estimular inovacoes tecnologicas de baixo carbono. o sbce funcionara no modelo de " cap - and - trade " : e estabelecido um limit\nestamos atentos aos desafios e as particularidades desse novo mercado. acompanharemos a criacao do novo orgao gestor e contribuiremos para a criacao de um arcabouco regulatorio robusto. alem disso, como e de praxe, a autarquia mantera o dialogo conti

In [58]:
prompt_contexto = f"""
Você é um especialista em mercado de carbono e regulação financeira, com profundo conhecimento das normas da CVM.
Sua tarefa é responder à consulta do usuário usando EXCLUSIVAMENTE as informações presentes no contexto fornecido.
Não utilize conhecimento externo.
Não invente informações ausentes do contexto.

Responda de forma:
- Objetiva;
- Precisa;
- Alinhada ao tom institucional da CVM;
- Com clareza regulatória;
- Sem mencionar que está usando um contexto.

Consulta do usuário:
'{query}'

Contexto (trechos extraídos do documento da CVM):
'{contexto_formatado}'
"""

resposta = modelo_google_langchain.invoke(prompt_contexto)

display(
    Markdown(resposta.content)
)

O mercado de carbono no Brasil é estruturado principalmente em torno do Sistema Brasileiro de Comércio de Emissões de Gases de Efeito Estufa (SBCE).

O SBCE é um ambiente regulado que opera sob um regime de controle de emissões de gases de efeito estufa (GEE), viabilizando a negociação de ativos que representam o direito de emissão, bem como a redução ou remoção de GEE. Ele funciona no modelo "cap-and-trade" e visa reduzir as emissões de GEE para promover o desenvolvimento sustentável, a equidade climática e estimular inovações tecnológicas de baixo carbono, contribuindo para o cumprimento dos compromissos internacionais do Brasil, como os assumidos no Acordo de Paris.

Os ativos integrantes do SBCE, como as Cotas Brasileiras de Emissão (CBEs) e os Certificados de Redução ou Remoção Verificada de Emissões (CRVEs), são instrumentos financeiros que representam o direito de emitir GEE ou a comprovação de sua redução/remoção. Esses ativos, assim como os créditos de carbono do mercado voluntário, são considerados valores mobiliários quando negociados no mercado financeiro e de capitais, conforme o Art. 14 da Lei nº 15.042/2024 e o Art. 2, inciso X, da Lei nº 6.385/76.

A Comissão de Valores Mobiliários (CVM) desempenha um papel essencial na regulação, supervisão e fiscalização desses ativos no mercado de capitais. A Autarquia atua para assegurar a transparência, integridade e segurança das operações, visando o desenvolvimento eficiente do mercado e a proteção ao investidor. Suas competências incluem a definição de regras informacionais para aprimorar a transparência, a adequada formação de preços e a fiscalização das operações para prevenir manipulação de mercado e fraudes, conforme as Leis nºs 15.042/2024 e 6.385/1976.

O mercado voluntário se caracteriza por transações espontâneas de créditos de carbono ou ativos integrantes do SBCE, onde os participantes negociam esses créditos para compensar suas próprias emissões. A Lei nº 15.042/2024 busca trazer maior padronização e segurança jurídica para os créditos gerados voluntariamente que forem utilizados no SBCE, os denominados CRVEs.

Para se manter informado sobre as atualizações e regulamentações da CVM sobre o mercado de carbono, recomenda-se acompanhar os comunicados publicados no site oficial da Autarquia e a página de finanças sustentáveis da CVM.